Implicit FDM error test with different parameters

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from math import sqrt, log, exp
from scipy import stats, linalg, interpolate
import numpy as np
import scipy
import random

from IPython.core.display import display, HTML
import json

In [2]:
def plot3D(X, Y, Z, height=600, xlabel = "Sigma", ylabel = "K", zlabel = "Option Price", initialCamera = None):
    options = {
        "width": "100%",
        "style": "surface",
        "showPerspective": True,
        "showGrid": True,
        "showShadow": False,
        "keepAspectRatio": True,
        "height": str(height) + "px"
    }
    if initialCamera:
        options["cameraPosition"] = initialCamera
    data = [ {"x": X[y,x], "y": Y[y,x], "z": Z[y,x]} for y in range(X.shape[0]) for x in range(X.shape[1]) ]
    visCode = r"""
       <link href="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis.min.css" type="text/css" rel="stylesheet" />
       <script src="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis.min.js"></script>
       <div id="pos" style="top:0px;left:0px;position:absolute;"></div>
       <div id="visualization"></div>
       <script type="text/javascript">
        var data = new vis.DataSet();
        data.add(""" + json.dumps(data) + """);
        var options = """ + json.dumps(options) + """;
        var container = document.getElementById("visualization");
        var graph3d = new vis.Graph3d(container, data, options);
        graph3d.on("cameraPositionChange", function(evt)
        {
            elem = document.getElementById("pos");
            elem.innerHTML = "H: " + evt.horizontal + "<br>V: " + evt.vertical + "<br>D: " + evt.distance;
        });
       </script>
    """
    htmlCode = "<iframe srcdoc='"+visCode+"' width='100%' height='" + str(height) + "px' style='border:0;' scrolling='no'> </iframe>"
    display(HTML(htmlCode))

In [3]:
def get_bs_price(s, sigma, t, r, k):
    y1 = (np.log(s/k) + (r + 0.5*(sigma**2))* t) / (sigma * np.sqrt(t))
    y2 = (np.log(s/k) + (r - 0.5*(sigma**2)) * t) / (sigma * np.sqrt(t))
    cdf1 = scipy.stats.norm.cdf(y1)
    cdf2 = scipy.stats.norm.cdf(y2)
    price = s * cdf1 - np.exp(-r * t) * k * cdf2
    return price

In [4]:
sigma = 0.2
K = 100;
r = 0.1;

T = np.linspace(0.01, 1, num=100)
S_0= np.linspace(K * 0.1, K*2, num=100)

CallPut = 0;

Gridsize = 30

X,Y = np.meshgrid(T,S_0)
BSprices = get_bs_price(Y, sigma, X, r, K)
FDMprices = np.zeros(BSprices.shape)

X, Y = np.meshgrid(np.linspace(0,100,100),np.linspace(0,100,100))
Z = BSprices
plot3D(X, Y, Z)

Implicit FDM class, remeber it is not the explicit one, too lazy to change it...

In [5]:
def FDM_Explicit_price(S_0, sigma, Tm, r, K, GridSize):
    TGrid = GridSize
    SGrid = GridSize
    Smax = 2 * S_0
    Tmax = Tm
    KGrid = np.linspace(0, Smax, SGrid)
    T = np.linspace(0, Tmax, TGrid)
    dt = T[1] - T[0]
    dS = KGrid[1] - KGrid[0]
    prices = np.zeros((SGrid, TGrid))
    
    if CallPut == 0:    
        prices[:,TGrid - 1] = np.maximum(KGrid - K, 0)
        prices[SGrid - 1,:] = (KGrid[SGrid - 1] - K) * np.exp((r) * T)
        prices[1,:] = 0
    elif CallPut == 1:
        prices[:,TGrid - 1] = np.maximum(K - KGrid, 0)
        prices[SGrid - 1,:] = 0
        prices[1,:] = (K - KGrid[SGrid - 1]) * np.exp((r) * T)
        
    a = 0.5 * dt*((r)/dS - sigma**2 * KGrid/(dS**2)) * KGrid
    b = 1 + dt * (sigma**2 * KGrid**2/(dS**2) + (r))
    c = - 0.5 * dt*((r)/dS + sigma**2 * KGrid/(dS**2)) * KGrid

    coeff = np.diag(a[2:SGrid],-1) + np.diag(b[1:SGrid]) + np.diag(c[1:SGrid-1],1)
    P, L, U = scipy.linalg.lu(coeff)
    Boundary = np.zeros((SGrid - 1,1))

    for j in range(TGrid - 2, 0, -1) :
        Boundary[0] = - a[2] * prices[1,j]
        prices[1:SGrid, j:j+1] = (linalg.inv(U).dot(linalg.inv(L).dot(prices[1:SGrid, (j + 1):(j + 2)] + Boundary)))

    impFDMPrice = interpolate.interp1d(KGrid, prices[:,1])(S_0)
    return impFDMPrice

In [6]:
%%time
for i in range(len(S_0)):
    for j in range(len(T)):
        FDMprices[i][j]=FDM_Explicit_price(S_0[i], sigma, T[j], r, K, Gridsize) 

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs


In [7]:
X, Y = np.meshgrid(np.linspace(0,100,100),np.linspace(0,100,100))
Z = FDMprices
plot3D(X, Y, Z)

In [8]:
X, Y = np.meshgrid(np.linspace(0,100,100),np.linspace(0,100,100))
Z = abs(FDMprices-BSprices)
print(np.mean(Z))
print(np.std(Z))
print(np.amax(Z))
plot3D(X, Y, Z)

In [17]:
T = 1
S_0 = 100;
r = 0.05;

sigma = np.linspace(0.01, 0.7, num=100)
K = np.linspace(S_0 * 0.1, S_0*2, num=100)

X,Y = np.meshgrid(K, sigma)
BSprices2 = get_bs_price(S_0, Y, T, r, X)
FDMprices2 = np.zeros(BSprices2.shape)

In [18]:
%%time
for i in range(len(sigma)):
    for j in range(len(K)):
        FDMprices2[i][j]=FDM_Explicit_price(S_0, sigma[i], T, r, K[j], Gridsize) 

CPU times: user 58.1 s, sys: 456 ms, total: 58.5 s
Wall time: 59.6 s


In [19]:
X, Y = np.meshgrid(np.linspace(0,100,100),np.linspace(0,100,100))
Z = FDMprices2
plot3D(X, Y, Z)

In [20]:
X, Y = np.meshgrid(np.linspace(0,100,100),np.linspace(0,100,100))
Z = abs(FDMprices2-BSprices2)
print(np.mean(Z))
print(np.std(Z))
print(np.amax(Z))
plot3D(X, Y, Z)

7.05569423605
8.84449521765
40.4069454385


In [14]:
S_0 = 100;
sigma=0.05
K=100

T = np.linspace(0.01, 2, num=100)
r = np.linspace(0.01, 0.5, num=100)

X,Y = np.meshgrid(r, T)
BSprices3 = get_bs_price(S_0, sigma, Y, X, K,)
FDMprices3 = np.zeros(BSprices3.shape)

In [15]:
%%time
for i in range(len(T)):
    for j in range(len(r)):
        FDMprices3[i][j]=FDM_Explicit_price(S_0, sigma, T[i], r[j], K, Gridsize)

CPU times: user 48.5 s, sys: 56.2 ms, total: 48.5 s
Wall time: 48.6 s


In [16]:
X, Y = np.meshgrid(np.linspace(0,100,100),np.linspace(0,100,100))
Z = FDMprices3
plot3D(X, Y, Z)

In [17]:
X, Y = np.meshgrid(np.linspace(0,100,100),np.linspace(0,100,100))
Z = abs(FDMprices3-BSprices3)
print(np.mean(Z))
print(np.std(Z))
print(np.amax(Z))
plot3D(X, Y, Z)

2.84235228942
6.5520576697
52.0757792922


28 Mar 2018